![](img/009.png)

**Представление для рейсов**

Над таблицей flights создано представление flights_v, содержащее дополнительную информацию:

- расшифровку данных об аэропорте вылета departure_airport, departure_airport_name, departure_city;
- расшифровку данных об аэропорте прибытия arrival_airport, arrival_airport_name, arrival_city;
- местное время вылета scheduled_departure_local, actual_departure_local;
- местное время прибытия scheduled_arrival_local, actual_arrival_local;
- продолжительность полета scheduled_duration, actual_duration.

**Представление для маршрутов**
Информация о маршруте (номер рейса, аэропорты отправления и назначения, модель самолета), не зависящую от конкретных дат рейсов, составляет представление routes. Кроме того, это представление показывает массив дней недели days_of_week, по которым совершаются по- леты, и плановую продолжительность рейса duration.

**Функция now**

Позиция «среза» данных на текущий момент сохранена в функции bookings.now. Ей можно пользоваться в запросах там, где в обычной жизни использовалась бы функция now

#### Выборка данных

выбираем из FROM столбцы, выводим их с именами AS

```sql
SELECT title AS course_title, hours FROM courses;
```

в данном случае все

```sql
SELECT * FROM таблицы;
```

DICTINCT удалит повторы

```sql
SELECT DISTINCT start_year FROM students;
```

условия фильтрации

```sql
SELECT * FROM courses WHERE hours > 45;
```

соединенеие двух таблиц (декартово произведенеие таблиц)

```sql
SELECT * FROM courses, exams;
```

через WHERE задано условие соединения

```sql
SELECT courses.title, exams.s_id, exams.score 
FROM courses, exams 
WHERE courses.c_no = exams.c_no;
```

другой вариант указать соединение - через JOIN

```sql
SELECT students.name, exams.score 
FROM students 
JOIN exams 
    ON students.s_id = exams.s_id 
    AND exams.c_no = 'CS305';
```

Добавление строк из левой таблицы, если их нет в правой
```sql
SELECT students.name, exams.score 
FROM students 
LEFT JOIN exams 
    ON students.s_id = exams.s_id 
    AND exams.c_no = 'CS305';
```

WHERE применяется к уже готовому соединению

```sql
SELECT students.name, exams.score 
FROM students 
LEFT JOIN exams 
    ON students.s_id = exams.s_id 
WHERE exams.c_no = 'CS305';
```
Результат оператора SELECT может быть использован для другого запроса (подзапрос)

```sql
SELECT name, 
    (SELECT score 
     FROM exams 
     WHERE exams.s_id = students.s_id 
     AND exams.c_no = 'CS305')
FROM students;
```

Скалярные подзапросы можно использовать в условиях фильтрации

```sql
SELECT * 
FROM exams 
WHERE (SELECT start_year 
       FROM students 
       WHERE students.s_id = exams.s_id) > 2014;
```

IN — проверяет, содержится ли значение в талице, возвращенное подзапросом

```sql
SELECT name, start_year 
FROM students 
WHERE s_id IN (SELECT s_id 
               FROM exams 
               WHERE c_no = 'CS305');
```

NOT IN

```sql
SELECT name, start_year 
FROM students 
WHERE s_id NOT IN (SELECT s_id 
                   FROM exams 
                   WHERE score < 5);
```

EXIST проверить, что возвращено хоть что-то

```sql
SELECT name, start_year 
FROM students 
WHERE NOT EXISTS (SELECT s_id 
                  FROM exams 
                  WHERE exams.s_id = students.s_id 
                  AND score < 5);
```

Возможно объявление псевдонимов

```sql
SELECT s.name, ce.score 
FROM students s 
JOIN (SELECT exams.* 
      FROM courses, exams 
      WHERE courses.c_no = exams.c_no 
      AND courses.title = 'Базы данных') ce
    ON s.s_id = ce.s_id;
```

или тоже самое без подапросов

```sql
SELECT s.name, e.score 
FROM students s, courses c, exams e 
WHERE c.c_no = e.c_no 
AND c.title = 'Базы данных' 
AND s.s_id = e.s_id;
```

соритровка

```sql
SELECT * 
FROM exams 
ORDER BY score, s_id, c_no DESC;
```

группеировка и агрегация

```sql
SELECT count(*), count(DISTINCT s_id), 
avg(score) 
FROM exams;
```

обновление данных

```sql
UPDATE courses 
SET hours = hours * 2 
WHERE c_no = 'CS301';
```

Удаление

```sql
DELETE FROM exams WHERE score < 5;
```

транзакция

```sql
BEGIN;

INSERT INTO groups(g_no, monitor) 
SELECT 'A-101', s_id 
FROM students 
WHERE name = 'Анна';

UPDATE students SET g_no = 'A-101';

COMMIT;
```

![](img/010.png)

In [1]:
import psycopg2
from data import acces

In [2]:
connection = psycopg2.connect(dbname=acces.DATABASE_NAME_DEMO,
                             user=acces.USER_NAME,
                             password=acces.PASS,
                             host='localhost',
                             port='5432')

In [21]:
def pg_query(query):
    
    with connection:
        with connection.cursor() as cursor:
            cursor.execute(query)
            answer = cursor.fetchone()
            
    return answer

Задача. Кто летел позавчера рейсом Москва (SVO) — Новосибирск (OVB) на месте 1A, и когда он забронировал свой билет?

In [24]:
query = """
SELECT 
    t.passenger_name,
    b.book_date
FROM
    bookings b
    JOIN tickets t 
        ON t.book_ref = b.book_ref
    JOIN boarding_passes bp 
        ON bp.ticket_no = t.ticket_no
    JOIN flights f 
        ON f.flight_id = bp.flight_id
WHERE f.departure_airport = 'SVO'
AND f.arrival_airport = 'OVB'
AND f.scheduled_departure::date = 
    bookings.now()::date - INTERVAL '2 day'
AND bp.seat_no = '1A';
"""

In [25]:
print(pg_query(query))

('SERGEY SCHERBAKOV', datetime.datetime(2017, 7, 28, 19, 39, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=120, name=None)))
